<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_Agnostic_Run.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Run Agnostic App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This path will run the executable you want, with some bells and whitles.

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.


## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
import json

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

import ipywidgets as widgets
from IPython.display import display, clear_output

Using local utilities library


---
## Connect to Tapis

In [3]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-23T05:46:57+00:00
 Token expires in: 0:56:45.811473
-- LOG IN SUCCESSFUL! --


---
## Configure App

In [4]:
app_id = 'agnostic-app-test'

#### List the app schema

In [5]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)

In [6]:
monitor_out = widgets.Output()
monitor_accordion = widgets.Accordion(children=[monitor_out])
monitor_accordion.selected_index = 0
monitor_accordion.set_title(0, f'Monitor Job')
display(monitor_accordion)

with monitor_out:
    OpsUtils.display_tapis_app_schema(appMetaData)

Accordion(children=(Output(),), selected_index=0, titles=('Monitor Job',))

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [7]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [8]:
tapisInputAll["maxMinutes"] = 6

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [9]:
print('app_id:',app_id)

app_id: agnostic-app-test


### OpenSees

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['Main Program'] = 'OpenSees'

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-express',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSees_opensees-express --



Accordion(children=(Output(),), selected_index=0, titles=('',))

### OpenSeesMP

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['Main Program'] = 'OpenSeesMP'

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 
tapisInput['Main Script'] = 'Ex1a_many.Canti2D.Push.mp.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-mp-s3',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesSP

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['Main Program'] = 'OpenSeesSP'

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 
tapisInput['Main Script'] = 'simpleSP.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-sp-s3',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['Main Program'] = 'python'

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy - mpi

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['Main Program'] = 'python'

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.mpi.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy - mpi4py

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['Main Program'] = 'python'

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.mpi4py.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------